In [35]:
# ============================================
# CÉLULA 2: AUTO-CONFIGURAÇÃO HAIKU 4.5
# ============================================
import boto3
import json
import pandas as pd
from datetime import datetime
from xhtml2pdf import pisa
import os
from pathlib import Path
import pickle

print("📚 Bibliotecas importadas!\n")

# Configurar Bedrock Runtime
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name="us-east-2"
)

print("✅ Cliente AWS Bedrock configurado!")
print("🔍 Tentando configurar Claude Haiku 4.5...\n")

# ==============================================================
# ESTRATÉGIA: Tentar múltiplas formas de acessar Haiku 4.5
# ==============================================================

MODEL_ID = None
MODEL_NAME = None

# TENTATIVA 1: Usar Application Inference Profile (cross-region)
print("📌 Tentativa 1: Application Inference Profile...")
try:
    # Para cross-region inference, usamos um formato especial
    # que roteia automaticamente para a região com capacidade
    test_model_id = 'us.anthropic.claude-haiku-4-5-20251001-v1:0'
    
    # Testar com uma chamada mínima
    test_config = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10,
        "messages": [{"role": "user", "content": "test"}]
    }
    
    response = bedrock_client.invoke_model(
        body=json.dumps(test_config),
        modelId=test_model_id,
        accept="application/json",
        contentType="application/json"
    )
    
    MODEL_ID = test_model_id
    MODEL_NAME = "Claude Haiku 4.5"
    print("   ✅ SUCESSO! Claude Haiku 4.5 disponível!\n")
    
except Exception as e:
    print(f"   ❌ Não disponível: {str(e)[:100]}...\n")

# TENTATIVA 2: Inference Profile ARN direto
if not MODEL_ID:
    print("📌 Tentativa 2: ARN de Foundation Model...")
    try:
        test_model_id = 'arn:aws:bedrock:us-east-2::foundation-model/anthropic.claude-haiku-4-5-20251001-v1:0'
        
        test_config = {
            "anthropic_version": "bedrock-2023-05-31",
            "max_tokens": 10,
            "messages": [{"role": "user", "content": "test"}]
        }
        
        response = bedrock_client.invoke_model(
            body=json.dumps(test_config),
            modelId=test_model_id,
            accept="application/json",
            contentType="application/json"
        )
        
        MODEL_ID = test_model_id
        MODEL_NAME = "Claude Haiku 4.5"
        print("   ✅ SUCESSO via ARN!\n")
        
    except Exception as e:
        print(f"   ❌ Não disponível: {str(e)[:100]}...\n")

# TENTATIVA 3: Listar inference profiles disponíveis
if not MODEL_ID:
    print("📌 Tentativa 3: Buscando Inference Profiles...")
    try:
        bedrock_config = boto3.client('bedrock', region_name="us-east-2")
        response = bedrock_config.list_inference_profiles()
        
        # Procurar por Haiku 4.5
        for profile in response.get('inferenceProfileSummaries', []):
            profile_id = profile.get('inferenceProfileId', '').lower()
            profile_arn = profile.get('inferenceProfileArn', '')
            
            if 'haiku' in profile_id and '4' in profile_id:
                # Testar este profile
                try:
                    test_config = {
                        "anthropic_version": "bedrock-2023-05-31",
                        "max_tokens": 10,
                        "messages": [{"role": "user", "content": "test"}]
                    }
                    
                    response = bedrock_client.invoke_model(
                        body=json.dumps(test_config),
                        modelId=profile_arn,
                        accept="application/json",
                        contentType="application/json"
                    )
                    
                    MODEL_ID = profile_arn
                    MODEL_NAME = "Claude Haiku 4.5"
                    print(f"   ✅ SUCESSO! Profile encontrado: {profile_id}\n")
                    break
                    
                except:
                    continue
        
        if not MODEL_ID:
            print("   ⚠️  Nenhum profile Haiku 4.5 funcional encontrado\n")
            
    except Exception as e:
        print(f"   ❌ Erro ao listar profiles: {str(e)[:100]}...\n")

# FALLBACK: Claude 3.5 Haiku (SEMPRE FUNCIONA)
if not MODEL_ID:
    print("📌 Fallback: Usando Claude 3.5 Haiku")
    MODEL_ID = 'us.anthropic.claude-3-5-haiku-20241022-v1:0'
    MODEL_NAME = "Claude 3.5 Haiku"
    print("   ✅ Configurado!\n")

# ==============================================================
# RESUMO DA CONFIGURAÇÃO
# ==============================================================

print("="*60)
print(f"🤖 MODELO FINAL: {MODEL_NAME}")
print(f"📍 ID: {MODEL_ID}")
print("="*60)

if "4.5" in MODEL_NAME or "4-5" in MODEL_ID:
    print("\n🎉 ÓTIMA NOTÍCIA! Claude Haiku 4.5 está funcionando!")
    print("\n💰 CUSTOS ESTIMADOS:")
    print("   • Por questão: ~$0.0008")
    print("   • 220 questões: ~$0.18 USD")
    print("\n⭐ QUALIDADE:")
    print("   • Near-frontier performance")
    print("   • Melhor modelo custo-benefício disponível")
    print("   • Ideal para análises em escala")
else:
    print("\n💡 Usando Claude 3.5 Haiku (excelente opção!)")
    print("\n💰 CUSTOS ESTIMADOS:")
    print("   • Por questão: ~$0.003")
    print("   • 220 questões: ~$0.66 USD")
    print("\n⭐ QUALIDADE:")
    print("   • Excelente performance")
    print("   • Comprovado e estável")
    print("   • Com nosso prompt otimizado = Ótimos resultados")

print("\n" + "="*60)
print("✅ CONFIGURAÇÃO COMPLETA E TESTADA!")
print("📌 Próximo: Execute Célula 3")
print("="*60)

📚 Bibliotecas importadas!

✅ Cliente AWS Bedrock configurado!
🔍 Tentando configurar Claude Haiku 4.5...

📌 Tentativa 1: Application Inference Profile...
   ✅ SUCESSO! Claude Haiku 4.5 disponível!

🤖 MODELO FINAL: Claude Haiku 4.5
📍 ID: us.anthropic.claude-haiku-4-5-20251001-v1:0

🎉 ÓTIMA NOTÍCIA! Claude Haiku 4.5 está funcionando!

💰 CUSTOS ESTIMADOS:
   • Por questão: ~$0.0008
   • 220 questões: ~$0.18 USD

⭐ QUALIDADE:
   • Near-frontier performance
   • Melhor modelo custo-benefício disponível
   • Ideal para análises em escala

✅ CONFIGURAÇÃO COMPLETA E TESTADA!
📌 Próximo: Execute Célula 3


In [36]:
# ============================================
# CÉLULA 3: PROMPT OTIMIZADO PARA HAIKU 4.5
# ============================================

# PROMPT ESPECIALMENTE DESENHADO PARA HAIKU 4.5
# (Mais estruturado e direto que para Sonnet)

PROMPT_HAIKU_45_OTIMIZADO = """Você é um Professor Especialista em Direito Administrativo com 15 anos de experiência analisando questões da FGV.

MISSÃO: Criar análise pedagógica COMPLETA e PROFISSIONAL desta questão.

╔══════════════════════════════════════════════════════════════╗
║  ESTRUTURA OBRIGATÓRIA DA ANÁLISE (SIGA RIGOROSAMENTE!)     ║
╚══════════════════════════════════════════════════════════════╝

1️⃣ IDENTIFICAÇÃO RÁPIDA
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🎯 Tema Central: [Identifique o tema ESPECÍFICO em 1 linha]
📜 Base Legal: [Lei/Artigo/Princípio aplicável]
🎓 Nível: [BÁSICO / INTERMEDIÁRIO / AVANÇADO]
🎪 Padrão FGV: [Tipo de enunciado usado]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


2️⃣ CONCEITOS-CHAVE (Memorize!)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Liste 4-5 conceitos essenciais com definições curtas:

✓ Conceito 1: [Nome] → [Definição em 1 linha]
✓ Conceito 2: [Nome] → [Definição em 1 linha]
✓ Conceito 3: [Nome] → [Definição em 1 linha]
✓ Conceito 4: [Nome] → [Definição em 1 linha]
✓ Conceito 5: [Nome] → [Definição em 1 linha]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


3️⃣ ANÁLISE DE CADA ALTERNATIVA (CRÍTICO!)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Para CADA alternativa, estruture assim:

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA A                                            │
└─────────────────────────────────────────────────────────────┘

Resumo: [Resuma a ideia em 1 linha]

Status: ❌ INCORRETA  (ou ✅ CORRETA)

📖 Fundamento Legal:
[Cite Art. X da Lei Y OU Princípio Z OU Doutrina/Jurisprudência]

💡 Análise Técnica:
[Explique DETALHADAMENTE por que está correta/incorreta]
[Cite o raciocínio jurídico aplicável]
[Identifique a palavra-chave que define o status]

⚠️ Pegadinha (se houver):
[Explique a armadilha da FGV nesta alternativa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA B                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA C                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA D                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────────────────────────────────────────────────────────┐
│ 🔤 ALTERNATIVA E                                            │
└─────────────────────────────────────────────────────────────┘

[Repita estrutura completa]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


4️⃣ GABARITO DEFINITIVO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
✅ GABARITO: {gabarito}

🎯 Por que esta é a ÚNICA correta:
[Reforce o raciocínio em 2-3 linhas]

📝 Resumo em 1 frase:
[Síntese da resposta correta]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


5️⃣ PEGADINHAS IDENTIFICADAS
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Liste TODAS as pegadinhas desta questão:

⚠️ Pegadinha 1: [Descreva]
⚠️ Pegadinha 2: [Descreva]
⚠️ Pegadinha 3: [Descreva se houver]

🚫 PALAVRAS-ARMADILHA encontradas:
[Liste palavras absolutistas: sempre, nunca, todo, nenhum, etc.]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


6️⃣ ESTRATÉGIA DE RESOLUÇÃO
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📚 COMO ESTUDAR:
- [Dica prática 1]
- [Dica prática 2]
- [Dica prática 3]

🔗 TEMAS RELACIONADOS:
- [Tema 1 para revisar]
- [Tema 2 para revisar]
- [Tema 3 para revisar]

⚖️ JURISPRUDÊNCIA (se aplicável):
[STF/STJ - Súmula/Tese relevante]

⏱️ TEMPO IDEAL DE RESOLUÇÃO:
Iniciante: [X min] | Experiente: [Y min]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━


7️⃣ ANÁLISE FINAL
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
📊 Dificuldade: [BÁSICO/INTERMEDIÁRIO/AVANÇADO]
Justificativa: [Por quê?]

🎯 Competência Avaliada:
[ ] Memorização de lei
[ ] Interpretação doutrinária
[ ] Aplicação prática
[ ] Raciocínio jurídico

📈 Frequência na FGV:
Este tema é: [Muito Cobrado / Médio / Raro]

💡 DICA FINAL:
[Uma dica de ouro para o concurseiro]
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╔══════════════════════════════════════════════════════════════╗
║  REGRAS OBRIGATÓRIAS (NÃO PULE NADA!)                       ║
╚══════════════════════════════════════════════════════════════╝

✓ Analise TODAS as 5 alternativas com a estrutura completa
✓ Cite fundamentos legais ESPECÍFICOS (Art. X da Lei Y)
✓ Identifique TODAS as pegadinhas
✓ Use linguagem TÉCNICA mas CLARA
✓ Seja DETALHADO (mínimo 3000 caracteres)
✓ Marque claramente ✅ CORRETA ou ❌ INCORRETA
✓ Mantenha a formatação com emojis e linhas

═══════════════════════════════════════════════════════════════

📊 DADOS DA QUESTÃO:

CONTEXTO:
- Banca: {banca}
- Ano: {ano}
- Órgão: {orgao}
- Prova: {prova}
- Disciplina: {disciplina}
- Assunto: {assunto}

ENUNCIADO:
{enunciado}

ALTERNATIVAS:
A) {alt_a}
B) {alt_b}
C) {alt_c}
D) {alt_d}
E) {alt_e}

GABARITO OFICIAL: {gabarito}

═══════════════════════════════════════════════════════════════

IMPORTANTE: Sua análise será avaliada por:
✓ Completude (analisou todas alternativas?)
✓ Profundidade (fundamentos legais citados?)
✓ Clareza (estrutura organizada?)
✓ Utilidade pedagógica (ensina de verdade?)

Comece agora com "1️⃣ IDENTIFICAÇÃO RÁPIDA":"""

print("✅ Prompt ULTRA-OTIMIZADO para Haiku 4.5 configurado!")
print("\n📋 Características do prompt:")
print("   ✓ Estrutura super clara e diretiva")
print("   ✓ Formato visual com emojis e linhas")
print("   ✓ Instruções explícitas e repetitivas")
print("   ✓ Templates para copiar")
print("   ✓ Validação de qualidade embutida")
print("\n" + "="*60)
print("📌 Próximo: Execute Célula 4")
print("="*60)

✅ Prompt ULTRA-OTIMIZADO para Haiku 4.5 configurado!

📋 Características do prompt:
   ✓ Estrutura super clara e diretiva
   ✓ Formato visual com emojis e linhas
   ✓ Instruções explícitas e repetitivas
   ✓ Templates para copiar
   ✓ Validação de qualidade embutida

📌 Próximo: Execute Célula 4


In [42]:
# ============================================
# CÉLULA 4: ANÁLISE OTIMIZADA (CORRIGIDA)
# ============================================

# Criar pasta de cache
CACHE_DIR = Path('cache_analises_haiku45')
CACHE_DIR.mkdir(exist_ok=True)

def validar_qualidade_analise(analise, questao_dict):
    """Valida qualidade da análise"""
    score = 0
    
    # 1. Tamanho
    tamanho = len(analise)
    if tamanho >= 3500:
        score += 20
    elif tamanho >= 2500:
        score += 15
    elif tamanho >= 1800:
        score += 10
    
    # 2. Análise de TODAS alternativas
    alternativas_presentes = sum([
        'ALTERNATIVA A' in analise.upper(),
        'ALTERNATIVA B' in analise.upper(),
        'ALTERNATIVA C' in analise.upper(),
        'ALTERNATIVA D' in analise.upper(),
        'ALTERNATIVA E' in analise.upper()
    ])
    score += alternativas_presentes * 12
    
    # 3. Marcação clara
    tem_marcacoes = analise.count('✅') + analise.count('❌')
    if tem_marcacoes >= 5:
        score += 10
    elif tem_marcacoes >= 3:
        score += 5
    
    # 4. Fundamentos legais
    termos_legais = [
        'art.' in analise.lower() or 'artigo' in analise.lower(),
        'lei' in analise.lower(),
        'constituição' in analise.lower() or 'cf' in analise.lower(),
        'princípio' in analise.lower()
    ]
    score += sum(termos_legais) * 2
    
    # 5. Estrutura organizada
    tem_estrutura = all([
        'IDENTIFICAÇÃO' in analise.upper() or '1️⃣' in analise,
        'CONCEITOS' in analise.upper() or '2️⃣' in analise,
        'GABARITO' in analise.upper()
    ])
    if tem_estrutura:
        score += 10
    
    # 6. Pegadinhas
    if 'pegadinha' in analise.lower() or '⚠️' in analise:
        score += 8
    
    return min(score, 100)


def analisar_questao_com_ia_otimizada(questao_dict, max_tentativas=2):
    """
    Análise otimizada com retry inteligente
    """
    
    questao_id = questao_dict.get('id_questao', 'unknown')
    cache_file = CACHE_DIR / f"{questao_id}.pkl"
    
    # Verificar cache
    if cache_file.exists():
        print(f"   💾 Cache encontrado!")
        with open(cache_file, 'rb') as f:
            return pickle.load(f)
    
    # Gerar análise
    for tentativa in range(1, max_tentativas + 1):
        try:
            if tentativa > 1:
                print(f"   🔄 Tentativa {tentativa}/{max_tentativas}...")
            
            # Montar prompt
            prompt_final = PROMPT_HAIKU_45_OTIMIZADO.format(
                banca=questao_dict.get('banca', 'N/A'),
                ano=questao_dict.get('ano', 'N/A'),
                orgao=questao_dict.get('orgao', 'N/A'),
                prova=questao_dict.get('prova', 'N/A'),
                disciplina=questao_dict.get('disciplina', 'N/A'),
                assunto=questao_dict.get('assunto', 'N/A'),
                enunciado=questao_dict.get('enunciado', 'N/A'),
                alt_a=questao_dict.get('alt_a', 'N/A'),
                alt_b=questao_dict.get('alt_b', 'N/A'),
                alt_c=questao_dict.get('alt_c', 'N/A'),
                alt_d=questao_dict.get('alt_d', 'N/A'),
                alt_e=questao_dict.get('alt_e', 'N/A'),
                gabarito=questao_dict.get('gabarito', 'N/A')
            )
            
            # ✅ CONFIGURAÇÃO CORRIGIDA (SEM top_p)
            config = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 5000,
                "temperature": 0.3,  # ✅ Só temperature (removido top_p)
                # "top_p": 0.9,  # ❌ REMOVIDO - Conflita com temperature
                "system": """Você é um Professor PhD em Direito Administrativo, especialista em análise pedagógica de questões FGV.

SUAS CARACTERÍSTICAS:
- 15 anos de experiência
- Taxa de aprovação dos alunos: 87%
- Especialidade: Identificar pegadinhas da FGV
- Método: Análises completas, técnicas e didáticas

SEUS PRINCÍPIOS:
1. SEMPRE analisar TODAS as 5 alternativas
2. SEMPRE citar fundamentos legais específicos
3. SEMPRE identificar pegadinhas
4. SEMPRE manter estrutura clara com emojis
5. SEMPRE ser técnico mas didático

Você NÃO aceita análises superficiais. Cada resposta deve ENSINAR profundamente.""",
                "messages": [
                    {
                        "role": "user",
                        "content": prompt_final
                    }
                ]
            }
            
            # Invocar Bedrock
            response = bedrock_client.invoke_model(
                body=json.dumps(config),
                modelId=MODEL_ID,
                accept="application/json",
                contentType="application/json"
            )
            
            # Extrair resposta
            resposta = json.loads(response['body'].read().decode('utf-8'))
            analise = resposta.get('content', [{}])[0].get('text', 'Erro')
            
            # Validar qualidade
            score = validar_qualidade_analise(analise, questao_dict)
            print(f"   📊 Score de qualidade: {score}/100")
            
            # Se qualidade boa, salvar e retornar
            if score >= 65:
                print(f"   ✅ Análise aprovada!")
                
                # Salvar no cache
                with open(cache_file, 'wb') as f:
                    pickle.dump(analise, f)
                
                return analise
            
            # Se qualidade baixa e ainda tem tentativas, ajustar
            elif tentativa < max_tentativas:
                print(f"   ⚠️  Qualidade abaixo do ideal, ajustando...")
                # Na próxima tentativa, aumentar temperature
                config["temperature"] = 0.4
                
        except Exception as e:
            print(f"   ❌ Erro: {str(e)[:100]}")
            if tentativa == max_tentativas:
                return f"❌ Erro após {max_tentativas} tentativas: {str(e)}"
            continue
    
    # Retornar última tentativa mesmo se não ideal
    with open(cache_file, 'wb') as f:
        pickle.dump(analise, f)
    return analise


print("✅ Sistema de análise CORRIGIDO criado!")
print("\n🔧 Correção aplicada:")
print("   ❌ Removido: top_p (conflitava com temperature)")
print("   ✅ Mantido: temperature (controle de criatividade)")
print("\n📊 Funcionalidades:")
print("   ✓ Cache inteligente")
print("   ✓ Retry automático")
print("   ✓ Validação de qualidade")
print("   ✓ Score mínimo: 65/100")
print("\n" + "="*60)
print("✅ PRONTO PARA USAR!")
print("📌 Próximo: Execute Célula 7 (teste) novamente")
print("="*60)

✅ Sistema de análise CORRIGIDO criado!

🔧 Correção aplicada:
   ❌ Removido: top_p (conflitava com temperature)
   ✅ Mantido: temperature (controle de criatividade)

📊 Funcionalidades:
   ✓ Cache inteligente
   ✓ Retry automático
   ✓ Validação de qualidade
   ✓ Score mínimo: 65/100

✅ PRONTO PARA USAR!
📌 Próximo: Execute Célula 7 (teste) novamente


In [38]:
# ============================================
# CÉLULA 5: FUNÇÃO DE GERAÇÃO DE PDF
# ============================================

def gerar_pdf_questao(questao_dict, analise_ia, output_path):
    """
    Gera PDF formatado profissionalmente
    """
    
    # Função para escapar HTML
    def escape_html(text):
        if pd.isna(text):
            return 'N/A'
        return (str(text)
                .replace('&', '&amp;')
                .replace('<', '&lt;')
                .replace('>', '&gt;')
                .replace('"', '&quot;')
                .replace("'", '&#39;'))
    
    # Converter análise para HTML
    analise_html = escape_html(analise_ia)
    analise_html = (analise_html
                   .replace('\n\n', '</p><p>')
                   .replace('\n', '<br/>')
                   .replace('**', '<b>')
                   .replace('**', '</b>'))
    
    # Gerar HTML das alternativas
    def gerar_html_alternativas(q):
        gabarito = str(q.get('gabarito', '')).strip().upper()
        alternativas_html = ""
        
        for letra in ['A', 'B', 'C', 'D', 'E']:
            texto = escape_html(q.get(f'alt_{letra.lower()}', 'N/A'))
            eh_correta = (letra == gabarito)
            
            if eh_correta:
                estilo = 'background: linear-gradient(135deg, #d4edda 0%, #c3e6cb 100%); border-left: 5px solid #28a745; padding: 15px; margin: 12px 0; border-radius: 6px; box-shadow: 0 2px 4px rgba(0,0,0,0.1);'
                label_estilo = 'background-color: #28a745; color: white; padding: 6px 14px; border-radius: 5px; font-weight: bold; margin-right: 12px; font-size: 13pt;'
                simbolo = '✅'
            else:
                estilo = 'background-color: #f8f9fa; border-left: 4px solid #dee2e6; padding: 15px; margin: 12px 0; border-radius: 6px;'
                label_estilo = 'background-color: #6c757d; color: white; padding: 6px 14px; border-radius: 5px; font-weight: bold; margin-right: 12px; font-size: 13pt;'
                simbolo = '⬜'
            
            alternativas_html += f'''
            <div style="{estilo}">
                <span style="{label_estilo}">{letra}</span>
                <span style="font-weight: bold; font-size: 12pt;">{simbolo}</span>
                <span style="font-size: 11pt;">{texto}</span>
            </div>
            '''
        
        return alternativas_html
    
    # Template HTML
    html_template = f'''
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8"/>
        <style>
            @page {{ size: a4 portrait; margin: 1.8cm; }}
            body {{ font-family: Arial; font-size: 11pt; line-height: 1.7; color: #2c3e50; }}
            .header {{ background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 28px; text-align: center; margin-bottom: 25px; border-radius: 10px; box-shadow: 0 4px 6px rgba(0,0,0,0.1); }}
            .header h1 {{ margin: 0 0 12px 0; font-size: 26pt; font-weight: bold; }}
            .metadata {{ font-size: 11pt; opacity: 0.95; line-height: 1.6; }}
            .metadata-badge {{ background-color: rgba(255,255,255,0.2); padding: 4px 12px; border-radius: 15px; margin: 3px; display: inline-block; }}
            .section {{ margin-bottom: 22px; }}
            .section-title {{ background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%); padding: 14px 20px; border-left: 6px solid #667eea; font-size: 15pt; font-weight: bold; margin-bottom: 16px; color: #2c3e50; border-radius: 5px; }}
            .info-grid {{ display: table; width: 100%; margin: 15px 0; }}
            .info-row {{ display: table-row; }}
            .info-label {{ display: table-cell; background-color: #f8f9fa; font-weight: bold; padding: 12px; border: 1px solid #dee2e6; width: 30%; }}
            .info-value {{ display: table-cell; padding: 12px; border: 1px solid #dee2e6; background-color: white; }}
            .enunciado {{ background: linear-gradient(135deg, #fffbea 0%, #fff8dc 100%); border: 3px solid #ffd93d; padding: 20px; margin: 18px 0; font-size: 12pt; line-height: 1.8; border-radius: 8px; }}
            .gabarito-box {{ background: linear-gradient(135deg, #28a745 0%, #20c997 100%); color: white; padding: 25px; text-align: center; font-size: 20pt; font-weight: bold; margin: 25px 0; border-radius: 10px; }}
            .analise-ia {{ background: linear-gradient(135deg, #e3f2fd 0%, #bbdefb 100%); border: 3px solid #2196f3; padding: 20px; margin: 20px 0; border-radius: 8px; line-height: 1.9; }}
            .footer {{ margin-top: 40px; padding-top: 20px; border-top: 3px solid #dee2e6; text-align: center; font-size: 9pt; color: #6c757d; }}
            .id-badge {{ background-color: #6c757d; color: white; padding: 5px 12px; border-radius: 5px; font-size: 9pt; font-weight: bold; display: inline-block; margin-top: 10px; }}
        </style>
    </head>
    <body>
        <div class="header">
            <h1>📋 ANÁLISE PEDAGÓGICA COMPLETA</h1>
            <div class="metadata">
                <span class="metadata-badge">🏛️ {escape_html(questao_dict.get('banca', 'N/A'))}</span>
                <span class="metadata-badge">📅 {escape_html(questao_dict.get('ano', 'N/A'))}</span>
                <span class="metadata-badge">🏢 {escape_html(questao_dict.get('orgao', 'N/A'))}</span>
                <br/><span style="font-size: 10pt;">{escape_html(questao_dict.get('prova', 'N/A'))}</span>
            </div>
        </div>
        
        <div class="section">
            <div class="section-title">📊 IDENTIFICAÇÃO</div>
            <div class="info-grid">
                <div class="info-row"><div class="info-label">📚 Disciplina</div><div class="info-value">{escape_html(questao_dict.get('disciplina', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">📖 Assunto</div><div class="info-value">{escape_html(questao_dict.get('assunto', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">🏛️ Banca</div><div class="info-value">{escape_html(questao_dict.get('banca', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">📅 Ano</div><div class="info-value">{escape_html(questao_dict.get('ano', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">🏢 Órgão</div><div class="info-value">{escape_html(questao_dict.get('orgao', 'N/A'))}</div></div>
                <div class="info-row"><div class="info-label">✅ Gabarito</div><div class="info-value"><b style="color: #28a745; font-size: 14pt;">{escape_html(questao_dict.get('gabarito', 'N/A'))}</b></div></div>
            </div>
            <div class="id-badge">ID: {escape_html(questao_dict.get('id_questao', 'N/A'))}</div>
        </div>
        
        <div class="section">
            <div class="section-title">📝 ENUNCIADO</div>
            <div class="enunciado">{escape_html(questao_dict.get('enunciado', 'N/A'))}</div>
        </div>
        
        <div class="section">
            <div class="section-title">🔤 ALTERNATIVAS</div>
            {gerar_html_alternativas(questao_dict)}
        </div>
        
        <div class="gabarito-box">✅ GABARITO: ALTERNATIVA {escape_html(questao_dict.get('gabarito', 'N/A'))}</div>
        
        <div class="section">
            <div class="section-title">🤖 ANÁLISE PEDAGÓGICA (Claude Haiku 4.5)</div>
            <div class="analise-ia"><p>{analise_html}</p></div>
        </div>
        
        <div class="footer">
            Análise gerada via AWS Bedrock<br/><b>Claude Haiku 4.5</b> - Sistema Otimizado v2.0<br/>{datetime.now().strftime('%d/%m/%Y às %H:%M:%S')}
        </div>
    </body>
    </html>
    '''
    
    # Gerar PDF
    try:
        with open(output_path, "wb") as pdf_file:
            pisa_status = pisa.CreatePDF(html_template, dest=pdf_file)
        
        if not pisa_status.err:
            print(f"   ✅ PDF salvo: {output_path}")
    except Exception as e:
        print(f"   ❌ Erro ao gerar PDF: {str(e)}")
        raise

print("✅ Função de geração de PDF criada!")
print("📌 Próximo: Execute Célula 6")                           

✅ Função de geração de PDF criada!
📌 Próximo: Execute Célula 6


In [ ]:
# ============================================
# CÉLULA 6: CARREGAR DATASET
# ============================================

CAMINHO_ARQUIVO = 'C:\welligton-projects-IA\RPA-PROJETO-1\questoes_Direito_Administrativo_T_FGVTJ - Copia.xlsx'

print(f"📂 Carregando dataset FGV...")
print("⏳ Aguarde...\n")

try:
    df = pd.read_excel(CAMINHO_ARQUIVO)
    
    print(f"✅ Dataset carregado!")
    print(f"📊 Total: {len(df)} questões\n")
    
    print("📈 ESTATÍSTICAS:")
    print(f"   🏛️  Banca: {df['Banca'].unique()[0]}")
    print(f"   📅 Anos: {sorted(df['Ano'].unique())}")
    print(f"   🏢 Órgãos: {df['Orgao'].nunique()} diferentes")
    print(f"   📚 Assuntos: {df['Assunto'].nunique()} temas")
    
    print("\n📊 Gabaritos:")
    for gab, count in df['Resposta_Correta'].value_counts().sort_index().items():
        porc = (count / len(df)) * 100
        print(f"   Alt {gab}: {count:3d} ({porc:5.1f}%)")
    
    print("\n" + "="*60)
    print("📌 Próximo: Execute Célula 7 para TESTAR")
    print("="*60)
    
except Exception as e:
    print(f"❌ Erro: {str(e)}")

📂 Carregando dataset FGV...
⏳ Aguarde...

✅ Dataset carregado!
📊 Total: 220 questões

📈 ESTATÍSTICAS:
   🏛️  Banca: FGV
   📅 Anos: [np.int64(2008), np.int64(2009), np.int64(2013), np.int64(2014), np.int64(2015), np.int64(2018), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024), np.int64(2025)]
   🏢 Órgãos: 22 diferentes
   📚 Assuntos: 12 temas

📊 Gabaritos:
   Alt A:  36 ( 16.4%)
   Alt B:  43 ( 19.5%)
   Alt C:  39 ( 17.7%)
   Alt D:  52 ( 23.6%)
   Alt E:  50 ( 22.7%)

📌 Próximo: Execute Célula 7 para TESTAR


<>:5: SyntaxWarning: invalid escape sequence '\w'
<>:5: SyntaxWarning: invalid escape sequence '\w'
C:\Users\welligton.costa.s\AppData\Local\Temp\ipykernel_19168\2134280326.py:5: SyntaxWarning: invalid escape sequence '\w'
  CAMINHO_ARQUIVO = 'C:\welligton-projects-IA\RPA-PROJETO-1\questoes_Direito_Administrativo_T_FGVTJ.xlsx'


In [43]:
# ============================================
# CÉLULA 7: TESTE COM 1 QUESTÃO
# ============================================

def testar_questao(dataframe, indice=0):
    """Testa com 1 questão"""
    
    print("🧪 TESTE - Processando 1 questão")
    print("="*70 + "\n")
    
    try:
        row = dataframe.iloc[indice]
        
        questao = {
            'id_questao': row.get('ID_Questao', 'N/A'),
            'banca': row.get('Banca', 'N/A'),
            'ano': row.get('Ano', 'N/A'),
            'orgao': row.get('Orgao', 'N/A'),
            'prova': row.get('Prova', 'N/A'),
            'disciplina': row.get('Disciplina', 'N/A'),
            'assunto': row.get('Assunto', 'N/A'),
            'enunciado': row.get('Enunciado', 'N/A'),
            'alt_a': row.get('Alternativa_A', 'N/A'),
            'alt_b': row.get('Alternativa_B', 'N/A'),
            'alt_c': row.get('Alternativa_C', 'N/A'),
            'alt_d': row.get('Alternativa_D', 'N/A'),
            'alt_e': row.get('Alternativa_E', 'N/A'),
            'gabarito': str(row.get('Resposta_Correta', 'N/A')).strip().upper()
        }
        
        print(f"📋 ID: {questao['id_questao']}")
        print(f"📅 {questao['ano']} | 🏢 {questao['orgao']}")
        print(f"📖 {questao['assunto'][:60]}...")
        print(f"✅ Gabarito: {questao['gabarito']}\n")
        
        print("🤖 Analisando com Claude Haiku 4.5...")
        print("⏳ Aguarde 15-25 segundos...\n")
        
        analise = analisar_questao_com_ia_otimizada(questao)
        
        if analise.startswith("❌"):
            raise Exception(analise)
        
        print(f"\n📏 Tamanho: {len(analise)} caracteres")
        print(f"📊 Preview (primeiros 600 chars):\n")
        print("="*70)
        print(analise[:600] + "...")
        print("="*70 + "\n")
        
        print("📄 Gerando PDF de teste...")
        output_path = f'TESTE_{questao["id_questao"]}.pdf'
        gerar_pdf_questao(questao, analise, output_path)
        
        print("\n" + "="*70)
        print("✅ TESTE CONCLUÍDO!")
        print(f"📁 Arquivo: {output_path}")
        print("="*70)
        print("\n📌 Se OK, execute Célula 8 para processar TODAS!")
        
        return questao, analise
        
    except Exception as e:
        print(f"❌ Erro: {str(e)}")
        import traceback
        traceback.print_exc()
        return None, None

# EXECUTAR TESTE
print("🚀 Iniciando teste...\n")
questao_teste, analise_teste = testar_questao(df, indice=0)

🚀 Iniciando teste...

🧪 TESTE - Processando 1 questão

📋 ID: Q3592889
📅 2025 | 🏢 TJ-SC
📖 Processo Administrativo - Lei nº 9.784 de 1999 e Lei nº 14.2...
✅ Gabarito: B

🤖 Analisando com Claude Haiku 4.5...
⏳ Aguarde 15-25 segundos...

   📊 Score de qualidade: 100/100
   ✅ Análise aprovada!

📏 Tamanho: 15481 caracteres
📊 Preview (primeiros 600 chars):

# 🎓 ANÁLISE PEDAGÓGICA COMPLETA - FUNDAMENTAÇÃO PER RELATIONEM EM PROCESSOS DISCIPLINARES

---

## 1️⃣ IDENTIFICAÇÃO RÁPIDA

🎯 **Tema Central:** Admissibilidade da fundamentação per relationem em processos administrativos disciplinares

📜 **Base Legal:** 
- Lei nº 9.784/1999 (Lei de Processo Administrativo Federal)
- Lei nº 14.210/2021 (Reforma do Processo Administrativo)
- Constituição Federal/1988 (Princípio do Devido Processo Legal)
- Súmula STJ nº 184

🎓 **Nível:** INTERMEDIÁRIO (exige conhecimento de jurisprudência STJ)

🎪 **Padrão FGV:** Questão de jurisprudência com pegadinha conceitual...

📄 Gerando PDF de teste...
   ✅ PDF salvo: 

In [20]:
# ============================================
# CÉLULA 8: PROCESSAR TODAS AS QUESTÕES
# ============================================

def processar_questoes(dataframe, pasta_saida='pdfs_fgv_direito_adm'):
    """
    Processa todas as questões: analisa com IA e gera PDFs
    
    Args:
        dataframe: DataFrame com as questões
        pasta_saida: Pasta onde salvar os PDFs
    """
    
    # Criar pasta de saída
    Path(pasta_saida).mkdir(exist_ok=True)
    
    total = len(dataframe)
    sucessos = 0
    erros = 0
    erros_detalhes = []
    
    print("="*70)
    print(f"🚀 PROCESSAMENTO EM LOTE - FGV DIREITO ADMINISTRATIVO")
    print("="*70)
    print(f"📊 Total de questões: {total}")
    print(f"📁 Pasta de saída: {pasta_saida}/")
    print(f"⏱️  Tempo estimado: ~{total * 25} segundos (~{total * 25 / 60:.1f} minutos)")
    print(f"💰 Custo estimado: ~${total * 0.003:.2f} USD")
    print("="*70 + "\n")
    
    confirmacao = input("⚠️  Digite 'SIM' para confirmar e iniciar: ")
    
    if confirmacao.upper() != 'SIM':
        print("❌ Processamento cancelado pelo usuário.")
        return
    
    print("\n🚀 Iniciando processamento...\n")
    
    for index, row in dataframe.iterrows():
        num_questao = index + 1
        
        try:
            print(f"{'='*70}")
            print(f"📌 QUESTÃO {num_questao}/{total} ({num_questao/total*100:.1f}%)")
            print(f"{'='*70}")
            
            # Montar dicionário da questão
            questao = {
                'id_questao': row.get('ID_Questao', 'N/A'),
                'banca': row.get('Banca', 'N/A'),
                'ano': row.get('Ano', 'N/A'),
                'orgao': row.get('Orgao', 'N/A'),
                'prova': row.get('Prova', 'N/A'),
                'disciplina': row.get('Disciplina', 'N/A'),
                'assunto': row.get('Assunto', 'N/A'),
                'enunciado': row.get('Enunciado', 'N/A'),
                'alt_a': row.get('Alternativa_A', 'N/A'),
                'alt_b': row.get('Alternativa_B', 'N/A'),
                'alt_c': row.get('Alternativa_C', 'N/A'),
                'alt_d': row.get('Alternativa_D', 'N/A'),
                'alt_e': row.get('Alternativa_E', 'N/A'),
                'gabarito': str(row.get('Resposta_Correta', 'N/A')).strip().upper()
            }
            
            print(f"📚 ID: {questao['id_questao']}")
            print(f"📅 {questao['ano']} | 🏢 {questao['orgao']}")
            print(f"📖 {questao['assunto'][:50]}...")
            print(f"✅ Gabarito: {questao['gabarito']}\n")
            
            # Analisar com IA
            print(f"🤖 Analisando com Claude...")
            analise = analisar_questao_com_ia(questao)
            
            if analise.startswith("❌"):
                raise Exception(analise)
            
            print(f"✅ Análise recebida ({len(analise)} caracteres)")
            
            # Gerar nome do arquivo
            assunto_limpo = str(questao['assunto']).replace(' ', '_').replace('/', '-')[:40]
            filename = f"Q{num_questao:03d}_{questao['ano']}_{questao['orgao']}_{assunto_limpo}_GAB{questao['gabarito']}.pdf"
            # Limpar caracteres especiais do filename
            filename = "".join(c for c in filename if c.isalnum() or c in ('_', '-', '.'))
            output_path = os.path.join(pasta_saida, filename)
            
            # Gerar PDF
            print(f"📄 Gerando PDF...")
            gerar_pdf_questao(questao, analise, output_path)
            
            sucessos += 1
            print(f"✅ Questão {num_questao} concluída!\n")
            
        except Exception as e:
            erros += 1
            erro_msg = f"Q{num_questao} ({questao.get('id_questao', 'N/A')}): {str(e)[:100]}"
            erros_detalhes.append(erro_msg)
            print(f"❌ ERRO: {str(e)[:150]}\n")
            continue
    
    # Resumo final
    print("\n" + "="*70)
    print("🎉 PROCESSAMENTO CONCLUÍDO!")
    print("="*70)
    print(f"✅ Sucessos: {sucessos}/{total} ({sucessos/total*100:.1f}%)")
    print(f"❌ Erros: {erros}/{total} ({erros/total*100:.1f}%)")
    print(f"📁 PDFs salvos em: {os.path.abspath(pasta_saida)}/")
    
    if erros > 0:
        print(f"\n⚠️  QUESTÕES COM ERRO:")
        for i, erro in enumerate(erros_detalhes[:10], 1):  # Mostrar só as 10 primeiras
            print(f"   {i}. {erro}")
        if len(erros_detalhes) > 10:
            print(f"   ... e mais {len(erros_detalhes)-10} erros")
    
    print("="*70)
    
    # Gerar arquivo de log
    log_path = os.path.join(pasta_saida, 'log_processamento.txt')
    with open(log_path, 'w', encoding='utf-8') as f:
        f.write(f"PROCESSAMENTO DE QUESTÕES FGV - DIREITO ADMINISTRATIVO\n")
        f.write(f"Data: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n")
        f.write(f"Total: {total} | Sucessos: {sucessos} | Erros: {erros}\n\n")
        if erros > 0:
            f.write("ERROS:\n")
            for erro in erros_detalhes:
                f.write(f"{erro}\n")
    
    print(f"📝 Log salvo em: {log_path}")

# EXECUTAR PROCESSAMENTO COMPLETO   
processar_questoes(df, pasta_saida='pdfs_fgv_direito_adm')

🚀 PROCESSAMENTO EM LOTE - FGV DIREITO ADMINISTRATIVO
📊 Total de questões: 220
📁 Pasta de saída: pdfs_fgv_direito_adm/
⏱️  Tempo estimado: ~5500 segundos (~91.7 minutos)
💰 Custo estimado: ~$0.66 USD



KeyboardInterrupt: Interrupted by user

In [ ]:
# ============================================
# CÉLULA 9: ESTATÍSTICAS DETALHADAS
# ============================================

import glob

def gerar_relatorio_completo(dataframe, pasta='pdfs_fgv_direito_adm'):
    """Gera relatório completo do processamento"""
    
    print("="*70)
    print("📊 RELATÓRIO COMPLETO DO PROCESSAMENTO")
    print("="*70 + "\n")
    
    # Contar PDFs gerados
    pdfs = glob.glob(f"{pasta}/*.pdf")
    total_pdfs = len(pdfs)
    
    print(f"📄 PDFs Gerados: {total_pdfs}/{len(dataframe)}")
    
    # Tamanho total
    if pdfs:
        tamanho_total = sum(os.path.getsize(pdf) for pdf in pdfs)
        tamanho_mb = tamanho_total / (1024 * 1024)
        print(f"💾 Tamanho total: {tamanho_mb:.2f} MB")
        print(f"📏 Tamanho médio: {tamanho_mb/total_pdfs:.2f} MB por PDF")
    
    # Análise por ano
    print(f"\n📅 QUESTÕES POR ANO:")
    for ano, count in sorted(dataframe['Ano'].value_counts().items(), reverse=True):
        porcentagem = (count / len(dataframe)) * 100
        barra = "█" * int(porcentagem / 2)
        print(f"   {ano}: {count:3d} questões ({porcentagem:5.1f}%) {barra}")
    
    # Análise por órgão (top 10)
    print(f"\n🏢 TOP 10 ÓRGÃOS:")
    for orgao, count in dataframe['Orgao'].value_counts().head(10).items():
        porcentagem = (count / len(dataframe)) * 100
        print(f"   {orgao:20s}: {count:3d} ({porcentagem:5.1f}%)")
    
    # Análise por assunto (top 10)
    print(f"\n📖 TOP 10 ASSUNTOS:")
    for assunto, count in dataframe['Assunto'].value_counts().head(10).items():
        assunto_curto = assunto[:50] + "..." if len(assunto) > 50 else assunto
        porcentagem = (count / len(dataframe)) * 100
        print(f"   {count:3d} ({porcentagem:4.1f}%): {assunto_curto}")
    
    # Distribuição de gabaritos
    print(f"\n✅ DISTRIBUIÇÃO DE GABARITOS:")
    gabaritos = dataframe['Resposta_Correta'].value_counts().sort_index()
    for gab, count in gabaritos.items():
        porcentagem = (count / len(dataframe)) * 100
        barra = "▓" * int(porcentagem)
        print(f"   Alternativa {gab}: {count:3d} ({porcentagem:5.1f}%) {barra}")
    
    # Teste qui-quadrado para verificar se distribuição é uniforme
    esperado = len(dataframe) / 5
    chi_quadrado = sum((count - esperado)**2 / esperado for count in gabaritos.values())
    print(f"\n📈 Análise estatística da distribuição:")
    print(f"   Valor esperado por alternativa: {esperado:.1f}")
    print(f"   Chi-quadrado: {chi_quadrado:.2f}")
    if chi_quadrado < 9.49:  # Valor crítico para 4 graus de liberdade, 95% confiança
        print(f"   ✅ Distribuição aproximadamente uniforme")
    else:
        print(f"   ⚠️  Distribuição não uniforme")
    
    print("\n" + "="*70)
    print(f"📁 Arquivos em: {os.path.abspath(pasta)}/")
    print("="*70)

# Gerar relatório
gerar_relatorio_completo(df, pasta='pdfs_fgv_direito_adm')